<a href="https://colab.research.google.com/github/sreejagundu/Image_Prediction_Cat_vs_Dog/blob/main/EICT_Problem_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [2]:
!kaggle datasets download -d tongpython/cat-and-dog

 99% 216M/218M [00:01<00:00, 103MB/s]
100% 218M/218M [00:01<00:00, 120MB/s]


In [3]:
import zipfile
zip_ref = zipfile.ZipFile('/content/cat-and-dog.zip','r')
zip_ref.extractall('/content')
zip_ref.close()

In [4]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import keras
import sklearn

In [5]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import MaxPooling2D

In [50]:
IMAGE_SIZE = 128
BATCH_SIZE = 32

# Images preprocessing before training
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory("/content/training_set/training_set",
                                                 target_size = (IMAGE_SIZE, IMAGE_SIZE),
                                                 batch_size = BATCH_SIZE,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory("/content/test_set/test_set",
                                            target_size = (IMAGE_SIZE, IMAGE_SIZE),
                                            batch_size = BATCH_SIZE,
                                            class_mode = 'binary')

Found 8005 images belonging to 2 classes.
Found 2023 images belonging to 2 classes.


In [51]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())

model.add(Dense(units= 64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(units = 1, activation = 'sigmoid'))

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [52]:
model.fit(x = training_set, validation_data = test_set, epochs = 5)

Epoch 1/5
251/251 [==============================] - 211s 828ms/step - loss: 0.6656 - accuracy: 0.5914 - val_loss: 0.6342 - val_accuracy: 0.6298
Epoch 2/5
251/251 [==============================] - 207s 825ms/step - loss: 0.6135 - accuracy: 0.6630 - val_loss: 0.5682 - val_accuracy: 0.7173
Epoch 3/5
251/251 [==============================] - 203s 807ms/step - loss: 0.5864 - accuracy: 0.6879 - val_loss: 0.5247 - val_accuracy: 0.7553
Epoch 4/5
251/251 [==============================] - 198s 790ms/step - loss: 0.5506 - accuracy: 0.7218 - val_loss: 0.4964 - val_accuracy: 0.7494
Epoch 5/5
251/251 [==============================] - 207s 825ms/step - loss: 0.5217 - accuracy: 0.7387 - val_loss: 0.5192 - val_accuracy: 0.7692


In [53]:
from keras.preprocessing import image
import matplotlib.pyplot as plt

In [54]:
print("The model class indices are: ", training_set.class_indices)

The model class indices are:  {'cats': 0, 'dogs': 1}


In [60]:
# testing on a single input
test_image = image.load_img("/content/test_set/test_set/dogs/dog.4088.jpg", target_size = (IMAGE_SIZE, IMAGE_SIZE))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
print(test_image)
result = model.predict(test_image)
if result[0][0] == 1:
    prediction = 'dog'
else:
    prediction = 'cat'



[[[[182.  39.  33.]
   [178.  36.  35.]
   [184.  27.  34.]
   ...
   [239. 230. 213.]
   [214. 205. 190.]
   [213. 210. 195.]]

  [[192.  31.  36.]
   [183.  33.  34.]
   [189.  36.  38.]
   ...
   [210. 203. 187.]
   [217. 209. 196.]
   [238. 228. 218.]]

  [[199.  34.  41.]
   [194.  34.  36.]
   [195.  46.  42.]
   ...
   [218. 212. 198.]
   [214. 208. 196.]
   [243. 230. 221.]]

  ...

  [[ 75. 107.  83.]
   [ 64.  80.  51.]
   [118. 122.  89.]
   ...
   [ 68.  95.  80.]
   [ 67.  85.  73.]
   [ 71.  90.  71.]]

  [[ 50.  86.  60.]
   [ 59.  80.  49.]
   [134. 141. 107.]
   ...
   [101. 128. 113.]
   [ 86. 104.  92.]
   [ 70.  92.  80.]]

  [[ 96. 135. 108.]
   [102. 129.  96.]
   [ 86. 102.  65.]
   ...
   [ 79. 106.  89.]
   [ 87. 105.  89.]
   [133. 158. 162.]]]]
1/1 [==============================] - 0s 25ms/step


In [61]:
print(prediction)

dog
